In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import os
%matplotlib inline

# load data

In [2]:
table = {}

for i in glob.glob("ms1_training/*"):
    
    df = pd.read_csv(i, index_col=0)
    binned = bin_time(df, rt_window=1, mz_window=1.0005079)
    table[i.replace("ms1_training/","").replace("_ms1.csv", "")] = binned
    

In [3]:
for i in glob.glob("ms1_external/*"):
    
    df = pd.read_csv(i, index_col=0)
    binned = bin_time(df, rt_window=1, mz_window=1.0005079)
    table[i.replace("ms1_external/","").replace("_ms1.csv", "")] = binned
    

In [4]:
df = pd.DataFrame(table).transpose()
df = df.apply(lambda x: x/x.max(), axis=1)
df

,0,1,2,3,4,5,6,7,8,9,...,1790,1791,1792,1793,1794,1795,1796,1797,1798,1799
PXD012749,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PXD017642,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PXD016156,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PXD014736,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PXD003409,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PXD000759,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PXD020329,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PXD009443,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PXD015280,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Setup training data

In [8]:
#test = pd.read_csv("../train_data/train_pairs_proteinID.txt")
pairs = pd.read_csv("train_pairs.txt")
pairs

,PID1,PID2,Jaccard_Index
0,PXD002434,PXD012611,0.000000
1,PXD018291,PXD014062,0.000000
2,PXD006539,PXD000050,0.000000
3,PXD018291,PXD002552,0.000000
4,PXD006539,PXD011152,0.000000
...,...,...,...
404545,PXD002087,PXD005353,0.497725
404546,PXD002212,PXD002211,0.514724
404547,PXD002081,PXD002046,0.531152
404548,PXD002081,PXD002085,0.542056


In [9]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(pairs, test_size=0.2, random_state=42)

In [10]:
train.sort_values(by=["Jaccard_Index"])

,PID1,PID2,Jaccard_Index
24597,PXD006236,PXD018300,0.000000
4874,PXD017386,PXD009683,0.000000
1450,PXD010237,PXD011085,0.000000
12451,PXD021198,PXD009595,0.000000
2536,PXD006008,PXD010944,0.000000
...,...,...,...
404542,PXD002081,PXD002082,0.448373
404543,PXD002087,PXD002046,0.452747
404544,PXD002081,PXD005353,0.457388
404546,PXD002212,PXD002211,0.514724


In [11]:
score = train['Jaccard_Index'].values
score

array([0.00465116, 0.        , 0.00389105, ..., 0.00345871, 0.0042735 ,
       0.00297619])

In [12]:
training_set = []

for index, row in train.iterrows():
    
#    if (row['PID1'] in df.index) and (row['PID2'] in df.index):
    
    testing = np.hstack(( np.array(df.loc[row['PID1']]), np.array(df.loc[row['PID2']])))
    
    training_set.append(testing)

In [13]:
len(training_set[0])

3600

In [14]:
training_set = np.array(training_set)
np.shape(training_set)

(323640, 3600)

# Test Set - Internal

In [15]:
testing_set = []

for index, row in test.iterrows():
    
#    if (row['PID1'] in df.index) and (row['PID2'] in df.index):
    
    testing = np.hstack(( np.array(df.loc[row['PID1']]), np.array(df.loc[row['PID2']])))
    
    testing_set.append(testing)

In [16]:
test_score = test["Jaccard_Index"].values
test_score

array([0.01551812, 0.03440539, 0.0377095 , ..., 0.        , 0.00108571,
       0.00575898])

In [17]:
testing_set = np.array(testing_set)
np.shape(testing_set)

(80910, 3600)

# Second Validation Set (External)

In [18]:
#test2 = pd.read_csv("../test_data/test_pairs_proteinID.txt")
test2 = pd.read_csv("test_pairs.txt")
test2

,PID1,PID2,Jaccard_Index
0,PXD005184,PXD016001,0.000000
1,PXD005184,PXD002548,0.000000
2,PXD018048,PXD017336,0.000000
3,PXD003105,PXD002548,0.000000
4,PXD016680,PXD017336,0.000000
...,...,...,...
2140,PXD007864,PXD016001,0.203302
2141,PXD015369,PXD006332,0.209389
2142,PXD009687,PXD006332,0.228904
2143,PXD029420,PXD025626,0.232863


In [19]:
testing_set2 = []

for index, row in test2.iterrows():
    
#    if (row['PID1'] in df.index) and (row['PID2'] in df.index):
    
    testing = np.hstack(( np.array(df.loc[row['PID1']]), np.array(df.loc[row['PID2']] )))
    
    testing_set2.append(testing)

In [20]:
testing_set2 = np.array(testing_set2)
np.shape(testing_set2)

(2145, 3600)

In [21]:
testing_score2 = test2['Jaccard_Index']

# Setup internal test set

# RF Training

In [26]:
from sklearn.ensemble import RandomForestRegressor

In [24]:
clf = RandomForestRegressor()

In [25]:
%time clf.fit(training_set, score)

CPU times: user 9h 14min 24s, sys: 18min 17s, total: 9h 32min 41s
Wall time: 1d 16h 5min 34s


RandomForestRegressor()

In [26]:
import pickle

# save
with open('RF_rt1','wb') as f:
    pickle.dump(clf,f)

In [27]:
import pickle

clf = pickle.load(open("RF_rt1.pkl", 'rb'))
clf

RandomForestRegressor()

# External Test Time

In [28]:
predicted_training = clf.predict(testing_set2)
#predicted_training = [i[0] for i in predicted_training]

In [29]:
len(predicted_training)

2145

In [30]:
np.array(predicted_training)

array([0.00946699, 0.01488032, 0.01821417, ..., 0.0932038 , 0.14199256,
       0.16670069])

In [31]:
np.array([i for i in testing_score2])

array([0.        , 0.        , 0.        , ..., 0.2289044 , 0.23286256,
       0.24710128])

In [32]:
from scipy import stats

stats.pearsonr(testing_score2, predicted_training)

(0.5609655814219301, 4.4174185654711806e-178)

In [33]:
sklearn.metrics.mean_squared_error(testing_score2, predicted_training)

0.0005314797474903332

In [34]:
sklearn.metrics.mean_absolute_error(testing_score2, predicted_training)

0.0166899630304733

# Internal Test

In [37]:
predicted_training = clf.predict(testing_set)

In [38]:
stats.pearsonr(test_score, predicted_training)

(0.9031735206875535, 0.0)

In [39]:
sklearn.metrics.mean_squared_error(test_score, predicted_training)

0.000205305334339457

In [40]:
sklearn.metrics.mean_absolute_error(test_score, predicted_training)

0.007674528130232326

# Training Validation

In [35]:
predicted_training = clf.predict(training_set)

In [36]:
stats.pearsonr(score, predicted_training)

(0.9889380062120839, 0.0)

In [37]:
sklearn.metrics.mean_squared_error(score, predicted_training)

2.8712282702162883e-05

In [38]:
sklearn.metrics.mean_absolute_error(score, predicted_training)

0.0028674419747418416